### puzzle 1 ###

Need the knot hash calculation from day 10:

In [1]:
def circular_subset(lst, start, length):
    subset = []
    for i in range(length):
        circ_pos = (start + i) % len(lst)
        subset.append(lst[circ_pos])
    return subset
def circular_fill_reversed(lst, start, subset):
    for i, val in enumerate(reversed(subset)):
        circ_pos = (start + i) % len(lst)
        lst[circ_pos] = val
def knot_hash(s):
    input_ascii = [ord(c) for c in s]
    input_ascii.extend([17, 31, 73, 47, 23])
    lst = list(range(256))
    pos = 0
    skip = 0
    for i in range(64):
        for length in input_ascii:
            subset = circular_subset(lst, pos, length)
            circular_fill_reversed(lst, pos, subset)
            pos = (pos + length + skip) % len(lst)
            skip += 1
    khash = []
    for i in range(16):
        block = lst[16*i:16*(i+1)]
        val = block[0]
        for x in block[1:]:
            val ^= x
        khash.append(val)
    return(''.join(['{0:02x}'.format(a) for a in khash]))
def knot_hash_bin(s):
    hex_hash = knot_hash(s)
    return ''.join(['{0:04b}'.format(int(c,16)) for c in hex_hash])

In [2]:
assert('28e7c4360520718a5dc811d3942cf1fd' == knot_hash("31,2,85,1,80,109,35,63,98,255,0,13,105,254,128,33"))

In [3]:
test_key = 'flqrgnkx'
puzzle_key = 'ljoxqyyw'

In [4]:
test_output = ['11010100',
               '01010101',
               '00001010',
               '10101101',
               '01101000',
               '11001001',
               '01000100',
               '11010110']
for i, res in enumerate(test_output):
    bin_hash = knot_hash_bin('{}-{}'.format(test_key, i))
    assert(bin_hash[:8] == test_output[i])

In [5]:
test_used = sum(knot_hash_bin('{}-{}'.format(test_key, i)).count('1') for i in range(128))
assert(8108 == test_used)

In [6]:
sum(knot_hash_bin('{}-{}'.format(puzzle_key, i)).count('1') for i in range(128))

8316

### puzzle 2 ###

In [7]:
import numpy as np

In [8]:
def knot_hash_grid(key):
    grid = np.zeros((128,128))
    for i in range(128):
        row_key = '{}-{}'.format(key, i)
        bin_hash = knot_hash_bin(row_key)
        grid[i,:] = [int(c) for c in bin_hash]
    return grid

In [9]:
test_grid = knot_hash_grid(test_key)
test_grid[:8, :8]

array([[ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.],
       [ 1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.],
       [ 0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.]])

In [10]:
seen = np.zeros((128,128), dtype=np.bool)

In [24]:
def follow(pos, grid, seen, region=None):
    if region is None:
        region = []
    region.append(pos)
    seen[pos] = True
    row,col = pos
    nrow,ncol = grid.shape
    dirs = []
    # east
    if col > 0:
        dirs.append((row, col-1))
    # west
    if col < ncol-1:
        dirs.append((row, col+1))
    # north
    if row > 0:
        dirs.append((row-1, col))
    # south
    if row < nrow-1:
        dirs.append((row+1, col))
    for newpos in dirs:
        if seen[newpos]:
            continue
        if grid[newpos] == 0:
            seen[newpos] = True
            continue
        seen, region = follow(newpos, grid, seen, region)
    return seen, region

In [25]:
grid8 = test_grid[:8, :8].copy()
grid8_seen = np.zeros((8,8), dtype=np.bool)
grid8_seen, reg = follow((0,0), grid8, grid8_seen)

In [32]:
print(grid8)
print('First region: ', reg)

[[ 1.  1.  0.  1.  0.  1.  0.  0.]
 [ 0.  1.  0.  1.  0.  1.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  1.  0.]
 [ 1.  0.  1.  0.  1.  1.  0.  1.]
 [ 0.  1.  1.  0.  1.  0.  0.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.]
 [ 1.  1.  0.  1.  0.  1.  1.  0.]]
First region:  [(0, 0), (0, 1), (1, 1)]


In [33]:
def find_regions(grid):
    nrow, ncol = grid.shape
    seen = np.zeros((nrow,ncol), dtype=np.bool)
    regions = []
    for row in range(nrow):
        for col in range(ncol):
            pos = (row, col)
            if seen[pos]:
                continue
            if grid[pos] == 0:
                seen[pos] = True
                continue
            # new region found if we get to here
            seen, region = follow(pos, grid, seen)
            regions.append(region)
    return regions
    

In [34]:
find_regions(grid8)

[[(0, 0), (0, 1), (1, 1)],
 [(0, 3), (1, 3)],
 [(0, 5), (1, 5)],
 [(1, 7)],
 [(2, 4), (3, 4), (3, 5), (4, 4), (5, 4)],
 [(2, 6)],
 [(3, 0)],
 [(3, 2), (4, 2), (4, 1), (5, 1), (5, 0), (6, 1), (7, 1), (7, 0)],
 [(3, 7)],
 [(5, 7)],
 [(6, 5), (7, 5), (7, 6)],
 [(7, 3)]]

In [35]:
test_regions = find_regions(test_grid)

In [38]:
assert(1242 == len(test_regions))

In [39]:
len(find_regions(knot_hash_grid(puzzle_key)))

1074